In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00


In [2]:
!pip install crypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 10.7 MB/s eta 0:00:00


In [5]:
import os
from google.colab import files
uploaded = files.upload()

Saving file.docx to file (1).docx
Saving private_key.pem to private_key.pem
Saving public_key.pem to public_key.pem


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes

# Function to chunk a file into smaller parts
def chunk_file(file_path, chunk_size=4096):
    with open(file_path, 'rb') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk

import rsa

# Generate a new pair of RSA keys with a key size of 2048 bits
public_key, private_key = rsa.newkeys(2048)

# Print the keys
print(f"public key: {public_key}")
print("\n")
print(f"private key: {private_key}")

# File paths
public_key_path = 'C:\\Users\\cavlc\\OneDrive\\Desktop\\^^\\Mathematical-Cipher\\Assignment-4\\public_key.pem'
private_key_path = 'C:\\Users\\cavlc\\OneDrive\\Desktop\\^^\\Mathematical-Cipher\\Assignment-4\\private_key.pem'

# Save the public key to a file
with open(public_key_path, 'wb') as f:
    f.write(public_key.save_pkcs1('PEM'))

# Save the private key to a file
with open(private_key_path, 'wb') as f:
    f.write(private_key.save_pkcs1('PEM'))

# Read and import the private key
with open(private_key_path, 'rb') as f:
    private_key = rsa.PrivateKey.load_pkcs1(f.read())

# Read and import the public key
with open(public_key_path, 'rb') as f:
    public_key = rsa.PublicKey.load_pkcs1(f.read())

print("Keys have been saved and reloaded successfully.")


# File paths
file_to_encrypt = '/content/file.docx'
output_dir = 'C:\\Users\\cavlc\\OneDrive\\Desktop\\^^\\Mathematical-Cipher\\Assignment-4\\chunks'

print(f"File to encrypt: {file_to_encrypt}")

# Read and import the public key
with open(public_key_path, 'rb') as key_file:
    public_key = RSA.import_key(key_file.read())

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

rsa_cipher = PKCS1_OAEP.new(public_key)
chunk_index = 0

for chunk in chunk_file(file_to_encrypt):
    # Generate AES key and cipher
    aes_key = get_random_bytes(16)
    aes_cipher = AES.new(aes_key, AES.MODE_EAX)

    # Encrypt the chunk with AES
    ciphertext, tag = aes_cipher.encrypt_and_digest(chunk)

    # Encrypt AES key with RSA
    encrypted_aes_key = rsa_cipher.encrypt(aes_key)

    # Create a filename for the chunk
    chunk_filename = os.path.join(output_dir, f'chunk_{chunk_index:04d}.bin')

    # Write the encrypted AES key and encrypted chunk to the file
    with open(chunk_filename, 'wb') as chunk_file:
        chunk_file.write(encrypted_aes_key)
        chunk_file.write(aes_cipher.nonce)
        chunk_file.write(tag)
        chunk_file.write(ciphertext)

    chunk_index += 1

print(f"File encrypted and chunks saved to {output_dir}.")


public key: PublicKey(17223122428105402794623110535629324719775283596522384752824829956351004618358826475579391305890098451103913118699901553813759864072221688436712866806707238283279571502430675034715264752659385054286939436389575864407478070732691030310802143824843717303055635224302988726790933125133851182600510974944771042046481366897742273563364531847626467323819553688935505794680244880768771398414753272682810453654142722807709057239681126731404534649219520858452552784190223219241205506766119007997860485125262281554307237730817287796569714595198676265213934626796961160357401649049871241064111999208477323624714343639591287794599, 65537)


private key: PrivateKey(17223122428105402794623110535629324719775283596522384752824829956351004618358826475579391305890098451103913118699901553813759864072221688436712866806707238283279571502430675034715264752659385054286939436389575864407478070732691030310802143824843717303055635224302988726790933125133851182600510974944771042046481366897742273563

In [7]:
# Install pycryptodome if not already installed
!pip install pycryptodome

from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
import os

# Correct file paths for keys and chunks
private_key_path = '/content/private_key.pem'
public_key_path = '/content/public_key.pem'
chunks_dir = '/content/drive/MyDrive/chunks'

# Check if the private key file exists
if not os.path.exists(private_key_path):
    print(f"Private key file not found at: {private_key_path}")
    raise FileNotFoundError(f"Private key file not found at: {private_key_path}")

# Read and import the private key
with open(private_key_path, 'rb') as f:
    private_key = RSA.import_key(f.read())

# Function to decrypt the chunks
def decrypt_chunks(chunks_dir, private_key):
    chunk_index = 0
    output_file_path = 'C:\\Users\\cavlc\\OneDrive\\Desktop\\^^\\Mathematical-Cipher\\Assignment-4\\decrypted_file.docx'

    with open(output_file_path, 'wb') as output_file:
        while True:
            chunk_filename = os.path.join(chunks_dir, f'chunk_{chunk_index:04d}.bin')

            if not os.path.exists(chunk_filename):
                break

            try:
                print(f"Decrypting chunk: {chunk_filename}")

                with open(chunk_filename, 'rb') as chunk_file:
                    # Read encrypted AES key
                    encrypted_aes_key = chunk_file.read(private_key.size_in_bytes())
                    print(f"Encrypted AES key size: {len(encrypted_aes_key)}")

                    # Read AES nonce
                    nonce = chunk_file.read(16)
                    print(f"Nonce size: {len(nonce)}")

                    # Read AES tag
                    tag = chunk_file.read(16)
                    print(f"Tag size: {len(tag)}")

                    # Read encrypted chunk
                    ciphertext = chunk_file.read()
                    print(f"Ciphertext size: {len(ciphertext)}")

                    # Decrypt AES key with RSA
                    rsa_cipher = PKCS1_OAEP.new(private_key)
                    aes_key = rsa_cipher.decrypt(encrypted_aes_key)
                    print("Decrypted AES key with RSA")

                    # Decrypt the chunk with AES
                    aes_cipher = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
                    decrypted_chunk = aes_cipher.decrypt_and_verify(ciphertext, tag)
                    print("Decrypted chunk with AES")

                    # Write decrypted chunk to output file
                    output_file.write(decrypted_chunk)
                    print(f"Wrote {len(decrypted_chunk)} bytes to output file")

            except Exception as e:
                print(f"Error decrypting chunk {chunk_index}: {str(e)}")
                break

            chunk_index += 1

    print(f"File decrypted and saved to {output_file_path}.")

# Decrypt the chunks and reassemble the file
decrypt_chunks(chunks_dir, private_key)


Decrypting chunk: /content/drive/MyDrive/chunks/chunk_0000.bin
Encrypted AES key size: 256
Nonce size: 16
Tag size: 16
Ciphertext size: 4096
Decrypted AES key with RSA
Decrypted chunk with AES
Wrote 4096 bytes to output file
Decrypting chunk: /content/drive/MyDrive/chunks/chunk_0001.bin
Encrypted AES key size: 256
Nonce size: 16
Tag size: 16
Ciphertext size: 4096
Decrypted AES key with RSA
Decrypted chunk with AES
Wrote 4096 bytes to output file
Decrypting chunk: /content/drive/MyDrive/chunks/chunk_0002.bin
Encrypted AES key size: 256
Nonce size: 16
Tag size: 16
Ciphertext size: 4096
Decrypted AES key with RSA
Decrypted chunk with AES
Wrote 4096 bytes to output file
Decrypting chunk: /content/drive/MyDrive/chunks/chunk_0003.bin
Encrypted AES key size: 256
Nonce size: 16
Tag size: 16
Ciphertext size: 1764
Decrypted AES key with RSA
Decrypted chunk with AES
Wrote 1764 bytes to output file
File decrypted and saved to C:\Users\cavlc\OneDrive\Desktop\^^\Mathematical-Cipher\Assignment-4\decr